# 📘 Text Generation With Markov Chains

This project implements a **Markov Chain based Text Generator** using **n-gram language modeling**.  
It demonstrates how a machine can "babble" text by learning how often tokens (words or characters) follow each other.

---

## 🔹 Environment & Configuration
We first import required libraries, fix randomness for reproducibility, and set **key configuration parameters**:

- `LEVEL`: `'word'` or `'char'`  
- `ORDER`: N-gram order (context size = ORDER-1)  
- `GENERATE_LEN`: How many tokens to generate  
- `SEED_TEXT`: Optional starting phrase  
- `HOLDOUT_FRACTION`: Fraction of dataset reserved for evaluation


In [ ]:
import random
import re
import math
import json
from collections import defaultdict, Counter
from google.colab import files

# For reproducibility
random.seed(42)
print("✅ Environment Ready")


✅ Environment Ready


In [ ]:
LEVEL = 'word'
ORDER = 3
GENERATE_LEN = 120
SEED_TEXT = ''
HOLDOUT_FRACTION = 0.1

assert LEVEL in {'word', 'char'}, "LEVEL must be 'word' or 'char'"
assert ORDER >= 2, "ORDER must be >= 2"

print(f"📌 LEVEL={LEVEL}, ORDER={ORDER}, GENERATE_LEN={GENERATE_LEN}")


📌 LEVEL=word, ORDER=3, GENERATE_LEN=120


## 🔹 Dataset Upload / Sample Fallback
We upload a `.txt` dataset. If no file is uploaded, we use a fallback story.  

- Run the cell → upload a file  
- Output shows the file name and character count  


In [ ]:
def load_text_from_upload():
    print("💡 Upload a .txt file")
    uploaded = files.upload()
    if not uploaded:
        return None, None
    fname = next(iter(uploaded.keys()))
    with open(fname, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()
    return fname, text

SAMPLE_TEXT = (
    "Once upon a time, in a land far away, there lived a curious student who loved to build models. "
    "They read books, collected stories, and stitched together patterns of words. "
    "One day, they discovered that by counting how often words follow other words, "
    "they could teach a machine to babble convincingly. "
    "Markov chains, simple yet powerful, became their favorite tool."
)

try:
    fname, raw_text = load_text_from_upload()
except Exception as e:
    print("No file uploaded:", e)
    fname, raw_text = None, None

if not raw_text:
    fname = "sample.txt"
    raw_text = SAMPLE_TEXT

print(f"📄 Loaded: {fname} | {len(raw_text):,} characters")


💡 Upload a .txt file


Saving alternate_sample_corpus.txt to alternate_sample_corpus.txt
📄 Loaded: alternate_sample_corpus.txt | 366 characters


## 🔹 Tokenization & Data Split
We **tokenize** the text into smaller units (words or characters) and split into **training** and **holdout** sets.

- Word-level tokenizer extracts words & punctuation  
- Char-level tokenizer splits text into characters  


In [ ]:
def simple_word_tokenize(text):
    return re.findall(r"[A-Za-z0-9']+|[.!?,;:\-\n]", text.lower())

def char_tokenize(text):
    return list(text)

if LEVEL == 'word':
    tokens = simple_word_tokenize(raw_text)
else:
    tokens = char_tokenize(raw_text)

print(f"🔹 {len(tokens):,} tokens")
print("Sample:", tokens[:20])


🔹 78 tokens
Sample: ['\n', 'once', 'upon', 'a', 'time', ',', 'in', 'a', 'land', 'far', 'away', ',', 'there', 'lived', 'a', 'curious', 'student', 'who', 'loved', 'to']


In [ ]:
split = int(len(tokens) * (1 - HOLDOUT_FRACTION))
train_tokens = tokens[:split]
holdout_tokens = tokens[split:]
print(f"Train: {len(train_tokens):,}, Holdout: {len(holdout_tokens):,}")


Train: 70, Holdout: 8


## 🔹 Build the N-gram Model
We build a **context → next token counter**.  

- For `ORDER=3`, context = 2 tokens → predicts the 3rd.  
- Stored in a dictionary of Counters.  

In [ ]:
def build_ngram_model(tokens, order):
    context_counts = defaultdict(Counter)
    for i in range(len(tokens) - order + 1):
        context = tuple(tokens[i:i+order-1])
        nxt = tokens[i+order-1]
        context_counts[context][nxt] += 1
    return context_counts

context_counts = build_ngram_model(train_tokens, ORDER)
print(f"📊 Unique contexts: {len(context_counts):,}")


📊 Unique contexts: 65


## 🔹 Text Generation
We generate text by:
1. Picking a starting context (from seed or random).  
2. Sampling next tokens based on learned probabilities.  
3. Repeating for `GENERATE_LEN` tokens.  


In [ ]:
def pick_next(counter):
    total = sum(counter.values())
    r = random.uniform(0, total)
    upto = 0
    for tok, cnt in counter.items():
        upto += cnt
        if upto >= r:
            return tok
    return random.choice(list(counter.keys()))

def choose_start_context(tokens, order, seed_text=''):
    if seed_text:
        seed_toks = simple_word_tokenize(seed_text) if LEVEL == 'word' else list(seed_text)
        if len(seed_toks) >= order - 1:
            return tuple(seed_toks[-(order-1):])
    start_idx = random.randint(0, len(tokens) - order)
    return tuple(tokens[start_idx:start_idx+order-1])

def generate_text(context_counts, order, length, seed_text=''):
    context = choose_start_context(train_tokens, order, seed_text)
    out = list(context)
    for _ in range(length):
        counter = context_counts.get(context)
        if not counter:
            context = random.choice(list(context_counts.keys()))
            counter = context_counts[context]
        nxt = pick_next(counter)
        out.append(nxt)
        context = tuple(out[-(order-1):])
    if LEVEL == 'word':
        text = " ".join(out)
        text = re.sub(r"\s+([.!?,;:])", r"\1", text)
        return text
    return "".join(out)

generated = generate_text(context_counts, ORDER, GENERATE_LEN, SEED_TEXT)
print("📝 Generated Text:\n", generated)


📝 Generated Text:
 a curious student who loved to build models. 
 they read books, collected stories, and stitched together patterns of words. 
 markov chains, simple yet of convincingly other day words away and there a convincingly words often words follow other words, 
 they read books, collected stories, and stitched together patterns of words. 
 they read books, collected stories, and stitched together patterns of words. 
 markov chains, simple yet away. a student words counting machine upon of one curious, yet models. 
 one day, they discovered that by counting how often words follow other words, 
 they read books, collected stories,


## 🔹 Model Evaluation
We evaluate with 3 metrics:
- **Coverage** → % of holdout contexts seen in training  
- **Avg branching factor** → Avg number of possible next tokens per context  
- **Cross-Entropy** → Average model surprise on holdout (lower = better)  


In [ ]:
def contexts_from(tokens, order):
    for i in range(len(tokens) - order + 1):
        yield tuple(tokens[i:i+order-1]), tokens[i+order-1]

holdout_pairs = list(contexts_from(holdout_tokens, ORDER))
seen_contexts = set(context_counts.keys())

coverage = sum(1 for ctx, _ in holdout_pairs if ctx in seen_contexts) / len(holdout_pairs) if holdout_pairs else float('nan')
avg_branching = sum(len(counter) for counter in context_counts.values()) / len(context_counts) if context_counts else float('nan')

def cross_entropy_bits(context_counts, pairs):
    total_bits = 0.0
    for ctx, nxt in pairs:
        counter = context_counts.get(ctx)
        if not counter:
            p = 1e-8
        else:
            V = len(counter)
            p = (counter.get(nxt, 0) + 1) / (sum(counter.values()) + V)
        total_bits += -math.log2(p)
    return total_bits / len(pairs) if pairs else float('nan')

ce = cross_entropy_bits(context_counts, holdout_pairs)

print(f"📈 Coverage: {coverage:.3f}")
print(f"📈 Avg branching factor: {avg_branching:.3f}")
print(f"📈 Cross-entropy: {ce:.3f} bits/token")


📈 Coverage: 0.000
📈 Avg branching factor: 1.046
📈 Cross-entropy: 26.575 bits/token
